In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd

import shapely
from shapely.ops import transform

In [2]:
# this is the modified geojson with reduced coordinate precision
input_file = r"E:\Projects\Omdena\_ProjectCanopy2\Code\GeoJSON_modification\image_Shifting_cultivation_6_edit.geojson"
output_file = r"E:\Projects\Omdena\_ProjectCanopy2\Code\GeoJSON_modification\image_Shifting_cultivation_6_dissolve.geojson"
output_file2 = r"E:\Projects\Omdena\_ProjectCanopy2\Code\GeoJSON_modification\image_Shifting_cultivation_6_simple.geojson"

In [3]:
gdf = gpd.read_file(input_file)

In [4]:
gdf.head()

,raster_val,new_val,Class,geometry
0,470.0,33,2,"POLYGON ((28.71070 0.28566, 28.71061 0.28566, ..."
1,479.0,34,2,"POLYGON ((28.71079 0.28566, 28.71070 0.28566, ..."
2,497.0,35,2,"POLYGON ((28.71088 0.28566, 28.71079 0.28566, ..."
3,563.0,40,2,"POLYGON ((28.71096 0.28566, 28.71088 0.28566, ..."
4,638.0,45,3,"POLYGON ((28.71105 0.28566, 28.71096 0.28566, ..."


In [5]:
raster_val = []
raster_val = pd.DataFrame(pd.unique(gdf["raster_val"].sort_values()), dtype = float, columns=['a'])
raster_val.reset_index(inplace=True)
raster_val.rename(columns={"index": "idx"},inplace=True)
raster_val.head()

,idx,a
0,0,194.0
1,1,196.0
2,2,200.0
3,3,201.0
4,4,203.0


In [6]:
gdf["new_val"] = 0

gdf = pd.merge(gdf, raster_val, left_on = 'raster_val', right_on = 'a', how='outer')

gdf.head()

,raster_val,new_val,Class,geometry,idx,a
0,470.0,0,2,"POLYGON ((28.71070 0.28566, 28.71061 0.28566, ...",270,470.0
1,470.0,0,2,"POLYGON ((28.73342 0.28557, 28.73333 0.28557, ...",270,470.0
2,470.0,0,2,"POLYGON ((28.72049 0.28548, 28.72040 0.28548, ...",270,470.0
3,470.0,0,2,"POLYGON ((28.73145 0.28521, 28.73136 0.28521, ...",270,470.0
4,470.0,0,2,"POLYGON ((28.73181 0.28476, 28.73172 0.28476, ...",270,470.0


In [7]:
gdf.new_val = (gdf["idx"]/max(gdf["idx"])*100).astype(int)
gdf.drop(['idx','a'], axis=1, inplace=True)
gdf.head()

,raster_val,new_val,Class,geometry
0,470.0,34,2,"POLYGON ((28.71070 0.28566, 28.71061 0.28566, ..."
1,470.0,34,2,"POLYGON ((28.73342 0.28557, 28.73333 0.28557, ..."
2,470.0,34,2,"POLYGON ((28.72049 0.28548, 28.72040 0.28548, ..."
3,470.0,34,2,"POLYGON ((28.73145 0.28521, 28.73136 0.28521, ..."
4,470.0,34,2,"POLYGON ((28.73181 0.28476, 28.73172 0.28476, ..."


In [8]:
import mapclassify


In [9]:
# Create a Natural Breaks classifier
mapclassify.NaturalBreaks(y=gdf.new_val, k=5)

NaturalBreaks           

    Interval       Count
------------------------
[  0.00,  12.00] | 87670
( 12.00,  21.00] | 39948
( 21.00,  37.00] | 16551
( 37.00,  57.00] |  9678
( 57.00, 100.00] |  4547

In [10]:
# Classify the data
gdf.Class = gdf.new_val.apply(mapclassify.NaturalBreaks(y=gdf.new_val, k=5))

# Let's see what we have
gdf.head()

,raster_val,new_val,Class,geometry
0,470.0,34,[2],"POLYGON ((28.71070 0.28566, 28.71061 0.28566, ..."
1,470.0,34,[2],"POLYGON ((28.73342 0.28557, 28.73333 0.28557, ..."
2,470.0,34,[2],"POLYGON ((28.72049 0.28548, 28.72040 0.28548, ..."
3,470.0,34,[2],"POLYGON ((28.73145 0.28521, 28.73136 0.28521, ..."
4,470.0,34,[2],"POLYGON ((28.73181 0.28476, 28.73172 0.28476, ..."


In [11]:
def flatten(input_array):
    result_array = []
    for element in input_array:
        #print(element[0])
        result_array.append(element[0]+1)
    return result_array

In [12]:
#results = flatten(gdf.Class)
results = pd.DataFrame(flatten(gdf.Class))
gdf.Class = results
gdf.head()

,raster_val,new_val,Class,geometry
0,470.0,34,3,"POLYGON ((28.71070 0.28566, 28.71061 0.28566, ..."
1,470.0,34,3,"POLYGON ((28.73342 0.28557, 28.73333 0.28557, ..."
2,470.0,34,3,"POLYGON ((28.72049 0.28548, 28.72040 0.28548, ..."
3,470.0,34,3,"POLYGON ((28.73145 0.28521, 28.73136 0.28521, ..."
4,470.0,34,3,"POLYGON ((28.73181 0.28476, 28.73172 0.28476, ..."


In [13]:
gdf_class = gdf.dissolve(by='Class')

In [14]:
gdf_class.drop(['raster_val', 'new_val'], axis=1, inplace=True)
gdf_class.reset_index(inplace=True)
gdf_class


,Class,geometry
0,1,"MULTIPOLYGON (((28.71096 0.25323, 28.71096 0.2..."
1,2,"MULTIPOLYGON (((28.71150 0.25252, 28.71150 0.2..."
2,3,"MULTIPOLYGON (((28.71150 0.25189, 28.71150 0.2..."
3,4,"MULTIPOLYGON (((28.71096 0.25171, 28.71105 0.2..."
4,5,"MULTIPOLYGON (((28.71070 0.25135, 28.71061 0.2..."


In [15]:
gdf_class2=gdf_class.explode(index_parts=True)

In [16]:
gdf_class2

Class                                           geometry
0 0        1  POLYGON ((28.71096 0.25323, 28.71096 0.25314, ...
  1        1  POLYGON ((28.71070 0.25350, 28.71070 0.25359, ...
  2        1  POLYGON ((28.71070 0.25656, 28.71061 0.25656, ...
  3        1  POLYGON ((28.71096 0.26662, 28.71096 0.26653, ...
  4        1  POLYGON ((28.71070 0.26752, 28.71070 0.26761, ...
...      ...                                                ...
4 495      5  POLYGON ((28.74878 0.28072, 28.74887 0.28072, ...
  496      5  POLYGON ((28.74932 0.28090, 28.74932 0.28081, ...
  497      5  POLYGON ((28.74816 0.28081, 28.74807 0.28081, ...
  498      5  POLYGON ((28.74851 0.28108, 28.74851 0.28099, ...
  499      5  POLYGON ((28.74950 0.28117, 28.74941 0.28117, ...

[9738 rows x 2 columns]

In [17]:
len(gdf)

158394

In [18]:
#save the file:
gdf_class2.to_file(output_file)

In [19]:
from shapely.geometry import Point, LineString
#EPSG:3857
#gdf_class2.to_file(output_file2)

gdf_class2 = gdf_class2.to_crs(3857)

In [20]:
#simplifying to a meter:
gdf_class2.geometry=gdf_class2.simplify(1)
gdf_class2 = gdf_class2.to_crs(4326)

In [21]:
gdf_class2.to_file(output_file2)

In [22]:
gdf_class2.tail()

Class                                           geometry
4 495      5  POLYGON ((28.74878 0.28072, 28.74887 0.28072, ...
  496      5  POLYGON ((28.74932 0.28090, 28.74932 0.28072, ...
  497      5  POLYGON ((28.74816 0.28081, 28.74807 0.28081, ...
  498      5  POLYGON ((28.74851 0.28108, 28.74851 0.28099, ...
  499      5  POLYGON ((28.74950 0.28117, 28.74941 0.28117, ...